# Creating a database with Postgres to store wealth data

### Functions to manage database

In [25]:
import psycopg2
import pandas as pd

def create_database():
    # Connect to default database
    try:
        conn = psycopg2.connect('host=127.0.0.1 dbname=postgres user=postgres password=root')
    except psycopg2.Error as e:
        print('Error connecting to database')
        print(e)
    
    conn.set_session(autocommit=True)
    cur = conn.cursor()

    # Create new database
    try:
        cur.execute('DROP DATABASE IF EXISTS wealth_database')
        cur.execute('CREATE DATABASE wealth_database')
    except psycopg2.Error as e:
        print('Error creating database')
        print(e)

    # Close connection to default database
    conn.close()

    # Connect to database
    conn = psycopg2.connect('host=127.0.0.1 dbname=wealth_database user=postgres password=root')
    cur = conn.cursor()

    conn.close()
    return cur, conn

def create_tables(cur, conn):
    for query in create_table_queries:
        cur.execute(query)
        conn.commit()

def drop_tabels(cur, conn):
    for query in drop_table_queries:
        cur.execute(query)
        conn.commit()


In [26]:
cur, conn = create_database()

### Cleaning data

In [36]:
acc_data = pd.read_csv('data/Wealth-AccountData.csv')

# Cleaning data
acc_data_clean = acc_data[['Country Code', 'Series Code', '1995 [YR1995]','2000 [YR2000]', '2005 [YR2005]', '2010 [YR2010]', '2014 [YR2014]']]
new_columns_data = {
    'Country Code': 'country_code'
    , 'Series Code': 'series_code'
    , '1995 [YR1995]': '1995'
    , '2000 [YR2000]': '2000'
    , '2005 [YR2005]': '2005'
    , '2010 [YR2010]': '2010'
    , '2014 [YR2014]': '2014'
}

acc_data_clean.rename(columns=new_columns_data,inplace=True)
acc_data_clean.head()

/tmp/ipykernel_42887/54510958.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  acc_data_clean.rename(columns=new_columns_data,inplace=True)


,country_code,series_code,1995,2000,2005,2010,2014
0,ALB,NW.HCA.TO,4.49E+10,4.36E+10,5.440000e+10,6.810000e+10,7.250000e+10
1,ALB,NW.HCA.PC,14072.79,14118.13,1.807230e+04,2.336286e+04,2.509002e+04
2,ALB,NW.HCA.FEMP.PC,3468.858,3424.007,2.222115e+03,2.889150e+03,2.957420e+03
3,ALB,NW.HCA.MEMP.PC,6507.931,6872.019,1.168935e+04,1.578992e+04,1.699138e+04
4,ALB,NW.HCA.FEMA.PC,4893.006,4695.071,2.886736e+03,3.613606e+03,3.719608e+03


In [34]:
acc_country = pd.read_csv('data/Wealth-AccountsCountry.csv')

# Cleaning data
acc_country_clean = acc_country[['Code', 'Short Name', 'Long Name', 'Region', 'Currency Unit']]
new_columns_country = {
    'Code': 'code'
    , 'Short Name': 'short_name'
    , 'Long Name': 'long_name'
    , 'Region': 'region'
    , 'Currency Unit': 'currency_unit'
}
acc_country_clean.rename(columns=new_columns_country, inplace=True)
acc_country_clean.head()

/tmp/ipykernel_42887/1371501140.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  acc_country_clean.rename(columns=new_columns_country, inplace=True)


,code,short_name,long_name,region,currency_unit
0,ALB,Albania,Republic of Albania,Europe & Central Asia,Albanian lek
1,ARG,Argentina,Argentine Republic,Latin America & Caribbean,Argentine peso
2,ARM,Armenia,Republic of Armenia,Europe & Central Asia,Armenian dram
3,AUS,Australia,Commonwealth of Australia,East Asia & Pacific,Australian dollar
4,AUT,Austria,Republic of Austria,Europe & Central Asia,Euro


In [43]:
acc_series = pd.read_csv('data/Wealth-AccountSeries.csv')

# Cleaning data
acc_series_clean = acc_series[['Code', 'Indicator Name', 'Topic', 'Periodicity', 'Unit of measure', 'Reference period']]
new_columns_series = {
    'Code': 'code'
    , 'Indicator Name': 'indicator_name'
    , 'Topic': 'topic'
    , 'Periodicity': 'periodicity'
    , 'Unit of measure': 'unit_of_measure'
    , 'Reference period': 'reference_period'
}
acc_series_clean.rename(columns=new_columns_series, inplace= True)
acc_country_clean.head()

/tmp/ipykernel_42887/42448928.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  acc_series_clean.rename(columns=new_columns_series, inplace= True)


,code,short_name,long_name,region,currency_unit
0,ALB,Albania,Republic of Albania,Europe & Central Asia,Albanian lek
1,ARG,Argentina,Argentine Republic,Latin America & Caribbean,Argentine peso
2,ARM,Armenia,Republic of Armenia,Europe & Central Asia,Armenian dram
3,AUS,Australia,Commonwealth of Australia,East Asia & Pacific,Australian dollar
4,AUT,Austria,Republic of Austria,Europe & Central Asia,Euro


### Creating tables at the database